Stars app
=

An example app for ingesting spatial data into a format readable by the Unreal Engine application at git@github.com:hamshifim/unreal-data-visualization.git. Both this repository and the repository holding the Unreal are submodules of this super repo: git@github.com:hamshifim/data-wielder.git and it holds the compatible versions of both repositories. Make sure you checkout the commit pointed to by data-wielder submodules for version alignment.

If it's your first time run the notebook cell by cell to while looking at the output directories; I would recommend using Sublime Text to view the directories intended for generated files.

For we are using CSV to import larger datasets to Unreal and are not aware of usage of a format including schema (e.g. parquet) supported by unreal. The last cell in this notebook generates the structs for the Unreal Engine application. After you are done you should build the code and import the tables, in Unreal and run the application using the instructions in the Unreal submodule.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyhocon import ConfigFactory as CF
from pyhocon import HOCONConverter

from pep_data.project import data_project_conf
from wielder.spark.util import set_spark_env
from wielder.unreal.util import to_unreal, UnrealTableType
from wielder.wield.project import default_conf_root, configure_project_module
from wielder.spark.util import columns_ranges
from wielder.spark.spatial import estimate_df_boundaries
from wielder.spark.util import color_by_range
from wielder.unreal.struct_gen import *

In [2]:
project_conf_dir = default_conf_root()
app = 'stars'
conf = data_project_conf(app=app)
configure_project_module(conf, app)

app_conf = conf[app]

/Users/gideonbar/dev/data-wielder/wdata/conf/bootstrap_env/docker/wield.conf
ERROR:root:'super' object has no attribute '__getattr__'


In [3]:
unreal_conf = app_conf.unreal_conf
dest = unreal_conf.conf

In [4]:
set_spark_env()
spark = SparkSession.builder.config("spark.driver.memory", "15g").appName(app).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/27 19:44:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
src = app_conf.src.stars_raw
dff = spark.read.csv(src, header=True)

In [6]:
dff.show(5)

+-----+---------+-------+----------+
|Index|        X|      Y|         Z|
+-----+---------+-------+----------+
|    1|      0.0|    0.0|       0.0|
|    2| 2197.405|0.03449|  41.77065|
|    3| 452.1092|0.03365|-160.08997|
|    4|3445.5278|0.30213| 2776.1497|
|    5| 828.3551|0.12476|-1056.1954|
+-----+---------+-------+----------+


In [7]:
def random_rgb():
    import random
    r = lambda: random.randint(0, 255)
    return '#%02X%02X%02X' % (r(), r(), r())

random_rgb_udf = F.udf(random_rgb)

In [8]:
def random_normalized_num():
    import random
    return random.uniform(0.01, 0.1)

random_normalized_num_udf = F.udf(random_normalized_num)

In [9]:
# cols = ['Index'] + app_conf.unreal_conf.base_columns
df = dff \
    .withColumn("x", F.col("x").cast("float")) \
    .withColumn("y", F.col("y").cast("float")) \
    .withColumn("z", F.col("z").cast("float")) \
    .withColumn("type", F.lit('stars')) \
    .withColumn("color", random_rgb_udf()) \
    .withColumn("size", random_normalized_num_udf()) \
    .withColumn("opacity", random_normalized_num_udf())

In [10]:
df.show(5)

+-----+---------+-------+----------+-----+-------+--------------------+--------------------+
|Index|        x|      y|         z| type|  color|                size|             opacity|
+-----+---------+-------+----------+-----+-------+--------------------+--------------------+
|    1|      0.0|    0.0|       0.0|stars|#821B72| 0.05186836976399395| 0.05186836976399395|
|    2| 2197.405|0.03449|  41.77065|stars|#1606E0| 0.06290146663828139| 0.06290146663828139|
|    3| 452.1092|0.03365|-160.08997|stars|#93C1B1| 0.05397169799968706| 0.05397169799968706|
|    4|3445.5278|0.30213| 2776.1497|stars|#4DCE09|0.030492069826705062|0.030492069826705062|
|    5| 828.3551|0.12476|-1056.1954|stars|#D22230|0.046524362496243796|0.046524362496243796|
+-----+---------+-------+----------+-----+-------+--------------------+--------------------+


In [11]:
# df.printSchema()

In [12]:
main_table = 'stars'
table = 'metadata'

In [13]:
unreal_conf.data_types[main_table].data_source

'/Users/gideonbar/dev/data-wielder/unreal-data-visualization/Content/Data/stars.csv'

In [14]:
dest = unreal_conf.data_types[main_table].data_source
to_unreal(df=df, dest=dest)

In [15]:
boundaries = estimate_df_boundaries(df)
boundaries

+-------------+---------+--------+----------------+---------+--------+------------+---------+--------+
|     center_x|    min_x|   max_x|        center_y|    min_y|   max_y|    center_z|    min_z|   max_z|
+-------------+---------+--------+----------------+---------+--------+------------+---------+--------+
|4.20556640625|-9370.511|9378.922|-195.77685546875|-9562.371|9170.817|44.626953125|-9613.174|9702.428|
+-------------+---------+--------+----------------+---------+--------+------------+---------+--------+

root
 |-- center_x: double (nullable = true)
 |-- min_x: float (nullable = true)
 |-- max_x: float (nullable = true)
 |-- center_y: double (nullable = true)
 |-- min_y: float (nullable = true)
 |-- max_y: float (nullable = true)
 |-- center_z: double (nullable = true)
 |-- min_z: float (nullable = true)
 |-- max_z: float (nullable = true)


{'center_x': 4.20556640625,
 'min_x': -9370.5107421875,
 'max_x': 9378.921875,
 'center_y': -195.77685546875,
 'min_y': -9562.37109375,
 'max_y': 9170.8173828125,
 'center_z': 44.626953125,
 'min_z': -9613.173828125,
 'max_z': 9702.427734375,
 'radius': 4851.2138671875}

In [16]:
unreal_conf.views.galaxy['boundaries'] = CF.from_dict(boundaries)


In [17]:
src = app_conf.src.stars_parameters
_col = "p_life"
params_df = spark.read.csv(src, header=True) \
    .withColumn('likelihood_of_life', F.col('likelihood_of_life').cast('float')) \
    .withColumn(_col, F.floor(F.col("likelihood_of_life")))

In [18]:
params_df.show(5)

+-----+---------+-----------+------------------+-------+-------------------+------------------+----------------+-------------+------+
|Index|star_name|galaxy_name|              size|  color|distance_from_earth|likelihood_of_life|num_surr_planets| type_of_star|p_life|
+-----+---------+-----------+------------------+-------+-------------------+------------------+----------------+-------------+------+
|    1|Komu kepo|  Cize loha|11.283134082358988|#83E792| 2387254243.6619678|          97.03046|               0|Main Sequence|    97|
|    2|Vuru goza|  Zuju vuhi|15.108297217469856|#2AC3C3|  3655900967.643971|          58.12317|              13|   Supergiant|    58|
|    3|Kina cowu|  Yazo keku| 3.823228120146334|#C6DDC3|  5519678016.621352|         46.228245|              16|  White Dwarf|    46|
|    4|Moxe copu|  Xixa xuyu|14.392850921395977|#0BF991|  1517002825.597455|          85.06668|               8|  White Dwarf|    85|
|    5|Fape zibu|  Kiqe xofu| 8.962635550725826|#834F39|  8754

In [19]:
# params_df.printSchema()
# params_df.columns

In [20]:
dest1 = unreal_conf.data_types[main_table].tables[table]['data_source']
dest1

'/Users/gideonbar/dev/data-wielder/unreal-data-visualization/Content/Data/stars_metadata.csv'

In [21]:
unreal_conf = to_unreal(params_df, dest1, unreal_conf, main_table, table, table_type=UnrealTableType.SECONDARY)

In [22]:
# unreal_conf

In [23]:
ranges = columns_ranges(params_df, [_col])
_min = ranges[f'min_{_col}']
_max = ranges[f'max_{_col}']

color_codes = color_by_range(_min, _max, cold=True)

unreal_conf.views.galaxy.color_maps[_col] = CF.from_dict(color_codes)
color_codes

{'0': '#00ff00',
 '1': '#00fc02',
 '2': '#00f905',
 '3': '#00f707',
 '4': '#00f40a',
 '5': '#00f20c',
 '6': '#00ef0f',
 '7': '#00ec12',
 '8': '#00ea14',
 '9': '#00e717',
 '10': '#00e519',
 '11': '#00e21c',
 '12': '#00e01e',
 '13': '#00dd21',
 '14': '#00da24',
 '15': '#00d826',
 '16': '#00d529',
 '17': '#00d32b',
 '18': '#00d02e',
 '19': '#00ce30',
 '20': '#00cb33',
 '21': '#00c836',
 '22': '#00c638',
 '23': '#00c33b',
 '24': '#00c13d',
 '25': '#00be40',
 '26': '#00bc42',
 '27': '#00b945',
 '28': '#00b648',
 '29': '#00b44a',
 '30': '#00b14d',
 '31': '#00af4f',
 '32': '#00ac52',
 '33': '#00aa55',
 '34': '#00a757',
 '35': '#00a45a',
 '36': '#00a25c',
 '37': '#009f5f',
 '38': '#009d61',
 '39': '#009a64',
 '40': '#009767',
 '41': '#009569',
 '42': '#00926c',
 '43': '#00906e',
 '44': '#008d71',
 '45': '#008b73',
 '46': '#008876',
 '47': '#008579',
 '48': '#00837b',
 '49': '#00807e',
 '50': '#007e80',
 '51': '#007b83',
 '52': '#007985',
 '53': '#007688',
 '54': '#00738b',
 '55': '#00718d',
 '

In [24]:
unreal_conf_j = HOCONConverter.to_json(unreal_conf, indent=4)
dest_j = unreal_conf.conf

with open(dest_j, 'w') as file:
    file.write(unreal_conf_j)

In [25]:
uconf = conf.unreal
solution_name = uconf.solution_name

if uconf.del_structs:
    delete_old_structs(solution_name, uconf.project_path)

# Define the path to your config file, written to location above
conf_path = uconf.conf_path
output_dir = uconf.struct_path

generate_structs(conf_path, output_dir)

Ignored col:  Index
Writing to:
/Users/gideonbar/dev/data-wielder/unreal-data-visualization/Source/Pepticom4D/StarsMetadataTempStruct.h
StarsMetadataTempStruct header and .cpp code written to:
/Users/gideonbar/dev/data-wielder/unreal-data-visualization/Source/Pepticom4D
